In [ ]:
!pip install pronouncing

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 11.2 MB/s eta 0:00:00
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6234 sha256=8bf612aef703c2fe45dd7ab0765853e48b4d71c58fdf20e87bcc602679516402
  Stored in directory: /root/.cache/pip/wheels/05/f6/1d/599c67da1fa48c086d8c49e8fc6bd5f05bc9fa66fb04bed5db
Successfully built pronouncing


In [ ]:
import librosa
import difflib
import pronouncing
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

def load_audio(file_path):
    speech, sr = librosa.load(file_path, sr=16000)
    return speech, sr

def transcribe_audio(file_path):
    speech, sr = load_audio(file_path)

    # Process the speech
    input_values = processor(speech, return_tensors="pt", sampling_rate=sr).input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # Decode the transcription
    transcription = processor.decode(predicted_ids[0])
    return transcription

def get_phonemes(word):
    # Get phonemes using the pronouncing library
    phoneme_list = pronouncing.phones_for_word(word.lower())
    if phoneme_list:
        return phoneme_list[0].split()
    return []

def compare_phonemes(reference_text, transcribed_text):
    ref_phonemes = [get_phonemes(word) for word in reference_text.split()]
    trans_phonemes = [get_phonemes(word) for word in transcribed_text.split()]

    errors = 0
    for ref, trans in zip(ref_phonemes, trans_phonemes):
        if ref != trans:
            errors += 1
            print(f"Pronunciation error: Expected {ref}, but got {trans}")

    return errors

def check_pronunciation(file_path, reference_text):
    transcribed_text = transcribe_audio(file_path)
    print(f"Transcribed Text: {transcribed_text}")

    errors = compare_phonemes(reference_text, transcribed_text)
    print(f"Total pronunciation errors: {errors}")

    return errors




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.maske

In [ ]:
audio_file = "/content/Recording 4.m4a"
reference_sentence = "This is an example sentence"
check_pronunciation(audio_file, reference_sentence)

<ipython-input-5-7aa4ee29207e>:13: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Transcribed Text: IS AN EXAMPLE SENTENC
Pronunciation error: Expected ['DH', 'IH1', 'S'], but got ['IH1', 'Z']
Pronunciation error: Expected ['IH1', 'Z'], but got ['AE1', 'N']
Pronunciation error: Expected ['AE1', 'N'], but got ['IH0', 'G', 'Z', 'AE1', 'M', 'P', 'AH0', 'L']
Pronunciation error: Expected ['IH0', 'G', 'Z', 'AE1', 'M', 'P', 'AH0', 'L'], but got []
Total pronunciation errors: 4


4

In [ ]:
audio_file = "/content/Recording.m4a"
reference_sentence = "as i told it was very good information for human"
check_pronunciation(audio_file, reference_sentence)

<ipython-input-3-2dfae378312c>:13: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Transcribed Text: AS I TOLD THAT IT WAS A VERY GOOD INFORMATION FOR HUMAN
Total pronunciation errors: 7
Score: 30.00 / 100
Pronunciation error: Expected ['IH1', 'T'], but got ['DH', 'AE1', 'T']
Pronunciation error: Expected ['W', 'AA1', 'Z'], but got ['IH1', 'T']
Pronunciation error: Expected ['V', 'EH1', 'R', 'IY0'], but got ['W', 'AA1', 'Z']
Pronunciation error: Expected ['G', 'UH1', 'D'], but got ['AH0']
Pronunciation error: Expected ['IH2', 'N', 'F', 'ER0', 'M', 'EY1', 'SH', 'AH0', 'N'], but got ['V', 'EH1', 'R', 'IY0']
Pronunciation error: Expected ['F', 'AO1', 'R'], but got ['G', 'UH1', 'D']
Pronunciation error: Expected ['HH', 'Y', 'UW1', 'M', 'AH0', 'N'], but got ['IH2', 'N', 'F', 'ER0', 'M', 'EY1', 'SH', 'AH0', 'N']


In [ ]:
# Example Usage
audio_file = "/content/Recording 4.m4a"
reference_sentence = "This is an example sentence"
check_pronunciation(audio_file, reference_sentence)

<ipython-input-5-7aa4ee29207e>:13: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Transcribed Text: THIS IS AN EXAMPLE SENTENCE
Total pronunciation errors: 0


0

In [ ]:
# Example Usage
audio_file = "/content/Recording 5.m4a"
reference_sentence = "This is an example sentence"
check_pronunciation(audio_file, reference_sentence)

<ipython-input-5-7aa4ee29207e>:13: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Transcribed Text: IIS AN EXAMPLE SENTENCE
Pronunciation error: Expected ['DH', 'IH1', 'S'], but got []
Pronunciation error: Expected ['IH1', 'Z'], but got ['AE1', 'N']
Pronunciation error: Expected ['AE1', 'N'], but got ['IH0', 'G', 'Z', 'AE1', 'M', 'P', 'AH0', 'L']
Pronunciation error: Expected ['IH0', 'G', 'Z', 'AE1', 'M', 'P', 'AH0', 'L'], but got ['S', 'EH1', 'N', 'T', 'AH0', 'N', 'S']
Total pronunciation errors: 4


4

In [ ]:
import librosa
import difflib
import pronouncing
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

# Load pre-trained model and processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")

def load_audio(file_path):
    # Load the audio file
    speech, sr = librosa.load(file_path, sr=16000)
    return speech, sr

def transcribe_audio(file_path):
    # Load audio
    speech, sr = load_audio(file_path)

    # Process the speech
    input_values = processor(speech, return_tensors="pt", sampling_rate=sr).input_values
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # Decode the transcription
    transcription = processor.decode(predicted_ids[0])
    return transcription

def get_phonemes(word):
    # Get phonemes using the pronouncing library
    phoneme_list = pronouncing.phones_for_word(word.lower())
    if phoneme_list:
        return phoneme_list[0].split()
    return []

def compare_phonemes(reference_text, transcribed_text):
    ref_phonemes = [get_phonemes(word) for word in reference_text.split()]
    trans_phonemes = [get_phonemes(word) for word in transcribed_text.split()]

    errors = 0
    error_details = []

    for ref, trans in zip(ref_phonemes, trans_phonemes):
        # Compare phonemes for each word
        if ref != trans:
            errors += 1
            error_details.append(f"Pronunciation error: Expected {ref}, but got {trans}")

    return errors, error_details

# Main function
def check_pronunciation(file_path, reference_text):
    remarks = transcribe_audio(file_path)
    print(f"Transcribed Text: {remarks}")

    # Compare with the reference text
    errors, error_details = compare_phonemes(reference_text, remarks)
    print(f"Total pronunciation errors: {errors}")

    # Calculate score
    total_words = len(reference_text.split())
    score = max(0, (total_words - errors) / total_words * 100)
    print(f"Score: {score:.2f} / 100")






/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a Be

In [ ]:
audio_file = "/content/Recording.m4a"
reference_sentence = "as i told it was very good information for human"
check_pronunciation(audio_file, reference_sentence)

<ipython-input-13-a4403c5d28ff>:13: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Transcribed Text: AS I TOLD THAT IT WAS A VERY GOOD INFORMATION FOR HUMAN
Total pronunciation errors: 7
Score: 30.00 / 100


In [ ]:
audio_file = "/content/Recording 3.m4a"
reference_sentence = "is an example sentence"
check_pronunciation(audio_file, reference_sentence)

<ipython-input-13-a4403c5d28ff>:13: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sr = librosa.load(file_path, sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Transcribed Text: IS AN EXAMPLE SENTENC
Total pronunciation errors: 1
Score: 75.00 / 100
